In [1]:
import sys
import numpy as np
import pandas as pd
import pygeohydro as gh
from pygeohydro import NWIS
import matplotlib.pyplot as plt

sys.path.append('../')

import sglib as sg
from sglib.verification.drought import get_drought_metrics, calculate_ssi_values
from sglib.utils.load import load_drb_reconstruction

from sglib.plotting.plot import plot_autocorrelation, plot_fdc_ranges, plot_flow_ranges
from sglib.plotting.drought import drought_metric_scatter_plot

from sglib.methods.nonparametric.kirsch_nowak import KirschNowakGenerator

In [2]:
Q = load_drb_reconstruction()

Q.head(2)

,cannonsville,pepacton,neversink,wallenpaupack,prompton,shoholaMarsh,mongaupeCombined,beltzvilleCombined,fewalter,merrillCreek,...,01436000,01433500,delMontague,01449800,01447800,delDRCanal,01463620,outletAssunpink,01470960,outletSchuylkill
1945-01-01,2182.881980,3049.283277,465.889412,547.218902,354.879704,102.372537,361.454234,62.050905,257.148891,62.894604,...,560.211141,434.819481,5819.910558,79.530825,200.786191,14280.028359,62.339044,162.218148,303.235746,7991.994247
1945-01-02,2365.621227,3062.214500,506.790219,764.847520,273.168187,145.409818,648.853734,118.134081,431.641388,89.318404,...,621.843822,901.203383,14796.024685,152.156677,453.815624,24072.036280,87.972290,266.062454,393.223713,10419.090255


In [4]:
# Initialize the generator
kn_gen = KirschNowakGenerator(Q)

# Preprocess the data
kn_gen.preprocessing()

# Fit the model
kn_gen.fit()

c:\Users\tjame\Desktop\Research\SGLib\examples\..\sglib\methods\nonparametric\kirsch_nowak.py:124: UserWarning: Matrix not positive definite, repairing...
  warnings.warn("Matrix not positive definite, repairing...")


Matrix repaired successfully after 15 iterations.
Matrix repaired successfully after 1 iterations.


In [5]:
synthetic_series = kn_gen.generate_single_series(n_years=10)


(30, 120)